In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.base import BaseEstimator

from scipy.optimize import fmin_cg
from scipy.optimize import fmin_l_bfgs_b
from scipy.optimize import fmin_bfgs

np.random.seed(42)

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.head()

,Timestamp,ID,Age?,Gender,Preferred Genres,Rating [Rogue One/Star Wars],Rating [Fight Club],Rating [The Lord of the Rings],Rating [Trolls],Rating [Despicable Me],...,Rating [The hangover],Rating [Captain America],Rating [The big Lebowski],Rating [Almost famous],Rating [The hunger games],Rating [Pulp Fiction],Rating [Sing ],Rating [500 days of Summer],Rating [Twilight],Rating [Lala Land]
0,2/9/2018 21:05:31,101,30-40,Male,"Comedy, Action, Romantic, Animation, Drama",NaN,5.0,5.0,NaN,NaN,...,3.0,NaN,NaN,NaN,2.0,NaN,NaN,5.0,5.0,5.0
1,2/9/2018 21:35:41,102,20-30,Male,"Comedy, Action, Drama",5.0,NaN,5.0,4.0,5.0,...,4.0,4.0,3.0,3.0,3.0,3.0,3.0,5.0,5.0,5.0
2,2/12/2018 12:25:41,103,20-30,Male,"Comedy, Action, Sci-Fi & Fantasy, Romantic, An...",5.0,5.0,5.0,NaN,2.0,...,2.0,5.0,5.0,5.0,2.0,5.0,NaN,NaN,3.0,5.0
3,2/12/2018 12:26:00,104,20-30,Male,"Action, Sci-Fi & Fantasy, Animation",4.0,5.0,4.0,NaN,5.0,...,NaN,4.0,NaN,NaN,NaN,NaN,3.0,4.0,2.0,3.0
4,2/12/2018 12:26:16,105,20-30,Female,"Comedy, Sci-Fi & Fantasy, Romantic",NaN,NaN,4.0,NaN,4.0,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.0


In [4]:
test = pd.read_csv('test.csv')

In [5]:
test.head()

,ID,Age?,Gender,Preferred Genres,Rating [Rogue One/Star Wars],Rating [Fight Club],Rating [The Lord of the Rings],Rating [Trolls],Rating [Despicable Me],Rating [Kubo],Rating [The hangover],Rating [Captain America],Rating [The big Lebowski],Rating [Almost famous],Rating [The hunger games],Rating [Pulp Fiction],Rating [Sing ],Rating [500 days of Summer],Rating [Twilight],Rating [Lala Land]
0,1,30-40,Female,"Comedy, Independent, Romantic",NaN,5.0,3.0,NaN,NaN,NaN,2.0,NaN,4.0,4.0,4.0,4.0,4.0,3.0,NaN,5.0
1,2,20-30,Female,"Comedy, Sci-Fi & Fantasy, Romantic",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,1.0
2,3,20-30,Male,"Comedy, Action, Sci-Fi & Fantasy",4.0,5.0,5.0,4.0,NaN,3.0,2.0,4.0,5.0,5.0,3.0,5.0,4.0,4.0,1.0,3.0
3,4,20-30,Male,Action,3.0,3.0,5.0,3.0,NaN,NaN,5.0,5.0,NaN,NaN,3.0,NaN,NaN,4.0,1.0,NaN
4,5,20-30,Male,"Comedy, Action, Sci-Fi & Fantasy",NaN,5.0,3.0,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,5.0,NaN,NaN,1.0,1.0


In [6]:
class MF:
    
    def __init__(self, n_factors=10, max_iter=20, reg=0.1,
                 use_bias=True, learning_rate=0.01,
                 solver='cg', shuffled=True):
        self.n_factors = n_factors
        self.max_iter = max_iter
        self.reg = reg
        self.learning_rate = learning_rate
        self.use_bias = use_bias
        self.solver = solver
        self.shuffled = shuffled
        
    def fit(self, ratings, y=None):
        """
        
        ratings: n_user * n_items sparse matrix
        
        """
        
        n_users, n_items = ratings.shape
        n_factors = self.n_factors
        reg = self.reg
        
        mask = ratings > 0
        #P = np.random.normal(0, 0.1, (n_users, n_factors))
        #Q = np.random.normal(0, 0.1, (n_items, n_factors))
        P = np.random.rand(n_users, n_factors)
        Q = np.random.rand(n_items, n_factors)
        
        # global bias
        if self.use_bias:
            global_mean = np.sum(ratings) / np.sum(mask)
        else:
            global_mean = 0
        bu = np.zeros(n_users) # user biases
        bi = np.zeros(n_items) # item biases
        
        if self.solver == 'sgd':
            
            lr = self.learning_rate
            P, Q, bu, bi = \
                self.sgd(ratings, mask, 
                         self.reg, self.learning_rate, self.max_iter, self.shuffled,
                         P, Q, bu, bi, self.use_bias, global_mean)
        else:
            P, Q, bu, bi = \
                self.fmin(ratings, mask, reg, self.max_iter, self.solver,
                          P, Q, bu, bi, self.use_bias, global_mean) 

        self.P_ = P
        self.Q_ = Q
        self.global_mean_ = global_mean
        self.bu_ = bu
        self.bi_ = bi
            
        return self
    
    def predict(self, X):
        pred = self.predict_full()
        pred[X.nonzero()] = X[X.nonzero()]
        
        return pred
    
    def predict_full(self):
        pred = np.dot(self.P_, self.Q_.T)
        
        if self.use_bias:
            pred = pred + self.global_mean_ + self.bu_[:,np.newaxis] + self.bi_[np.newaxis:, ]
            
        return pred
    
    def score(self, X, y=None):
        pred = self.predict_full()
        pred = pred[X.nonzero()].flatten()
        X = X[X.nonzero()].flatten()
        return np.sqrt(mean_squared_error(pred, X))
    
        
    def sgd(self, ratings, mask, reg, lr, epochs, shuffled, 
            P, Q, bu, bi, use_bias, global_mean):
                
        user_id, item_id = np.nonzero(ratings)
        all_ratings = np.array([(u, i) for u, i in zip(user_id, item_id)])
        idx = np.arange(len(all_ratings))

        for epoch in range(epochs):    
            #if shuffled:
            #    np.random.shuffle(idx)
            shuffled_ratings = all_ratings[idx]
            
            for u, i in shuffled_ratings:
                
                # compute current error
                dot = 0
                for f in range(self.n_factors):
                    dot += Q[i, f] * P[u, f]
                #dot = np.dot(P[u, :], Q[i, :])
                #err = (global_mean + bu[u] + bi[i] + dot) - ratings[u, i]
                err = ratings[u, i] - (global_mean + bu[u] + bi[i] + dot)
                
                # update biases
                if use_bias:
                    bu[u] += lr * (err - reg * bu[u])
                    bi[i] += lr * (err - reg * bi[i])

                # update factors                
                for f in range(self.n_factors):
                    puf = P[u, f]
                    qif = Q[i, f]
                    P[u, f] += lr * (err * qif - reg * puf)
                    Q[i, f] += lr * (err * puf - reg * qif)
                #puf = P[u, :]
                #qif = Q[i, :]
                #P[u, :] += lr * (err * qif - reg * puf)
                #Q[i, :] += lr * (err * puf - reg * qif)
        
        return P, Q, bu, bi
    
    def fmin(self, ratings, mask, reg, max_iter, solver,
           P, Q, bu, bi, use_bias, global_mean):
        
        def f(x0, *args):
            cost, grad = self.cost_function(x0, *args)
            
            return cost
            
        def fprime(x0, *args):
            cost, grad = self.cost_function(x0, *args)
            
            return grad.ravel()
                    
        n_users, n_items = ratings.shape
        n_factors = self.n_factors
    
        args = (ratings, mask, n_users, n_items, n_factors, 
                reg, use_bias, global_mean)
    
        x0 = np.hstack((P.ravel(), Q.ravel(), bu.ravel(), bi.ravel()))
        
        
        P_sz = n_users * n_factors
        Q_sz = n_items * n_factors
        if solver == 'cg':
            x = fmin_cg(f, x0, fprime=fprime, args=args, maxiter=self.max_iter, disp=0)
        elif solver == 'lbfgs':
            bounds = [(0, None)] * len(x0)
            #p_bounds = [(0, None)] * P_sz
            #q_bounds = [(0, None)] * Q_sz
            #bu_bounds = [(-2, 2)] * n_users
            #bi_bounds = [(-2, 2)] * n_items
            #bounds = p_bounds + q_bounds + bu_bounds + bi_bounds
            x, f, d = fmin_l_bfgs_b(f, x0, fprime=fprime, args=args, maxiter=self.max_iter, bounds=bounds)
        elif solver == 'bfgs':
            x = fmin_bfgs(f, x0, fprime=fprime, args=args, maxiter=self.max_iter)
 
        P = x[:P_sz].reshape(n_users, n_factors)
        Q = x[P_sz: (P_sz + Q_sz)].reshape(n_items, n_factors)
        bu = x[(P_sz + Q_sz): (P_sz + Q_sz + n_users)].reshape(n_users)
        bi = x[-n_items:].reshape(n_items)
        
        return P, Q, bu, bi
        
    def cost_function(self, x, ratings, mask, n_users, n_items, n_factors, 
                      reg, use_bias, global_mean):
        
        P_sz = n_users * n_factors
        Q_sz = n_items * n_factors
        # user features matrix
        P = x[:P_sz].reshape([n_users, n_factors])
    
        # item features matrix
        Q = x[P_sz: (P_sz + Q_sz)].reshape([n_items, n_factors])                

        # bias
        bu = x[(P_sz + Q_sz): (P_sz + Q_sz + n_users)].reshape(n_users)
        bi = x[-n_items:].reshape(n_items)
                
        dot = np.dot(P, Q.T)
        pred = dot + global_mean + bu[:,np.newaxis] + bi[np.newaxis:,]
        err = (pred - ratings) * mask
           
        cost_e = np.linalg.norm(err)
        cost_p = reg * np.linalg.norm(Q)
        cost_q = reg * np.linalg.norm(P)
            
        grad_p = np.dot(err, Q) + (reg * P)
        grad_q = np.dot(err.T, P) + (reg * Q)
        
        if use_bias:    
            cost_bu = reg * np.linalg.norm(bu)
            cost_bi = reg * np.linalg.norm(bi)
            grad_bu = np.sum(err, axis=1) + reg * bu
            grad_bi = np.sum(err, axis=0) + reg * bi
        else:
            cost_bu = 0
            cost_bi = 0
            grad_bu = 0 * bu
            grad_bi = 0 * bi
            
        cost = cost_e + cost_p + cost_q + cost_bu + cost_bi
        grad = np.hstack((grad_p.ravel(), grad_q.ravel(), 
                          grad_bu.ravel(), grad_bi.ravel()))
        
        return 0.5 * cost, grad

In [7]:
# def test_valid_rating_split(ratings, n_splits=3):

#     user_id, item_id = np.nonzero(movie_ratings)
#     all_ratings = np.array([(u, i) for u, i in zip(user_id, item_id)])
    
#     idx = np.arange(len(all_ratings))
#     np.random.shuffle(idx)
#     all_ratings = all_ratings[idx]
#     skf = StratifiedKFold(n_splits)
    
#     for train_idx, valid_idx in skf.split(all_ratings[:, 0], all_ratings[:, 0]):
#         train_rating = np.zeros_like(movie_ratings)
#         valid_rating = np.zeros_like(movie_ratings)
    
#         for u, i in all_ratings[train_idx]:
#             train_rating[u, i] = movie_ratings[u, i]
#         for u, i in all_ratings[valid_idx]:
#             valid_rating[u, i] = movie_ratings[u, i]
#         yield train_rating, valid_rating

In [8]:
# param_grid = {
#     'n_factors': [10, 12, 20],
#     'solver': ['sgd', 'lbfgs'],
#     'max_iter': [60, 100],
#     'reg': [0.1, 1],
#     'learning_rate': [0.01]
# }

# rating_columns = [col for col in train.columns if col.startswith('Rating')]
# data = test
# movie_ratings = movie_ratings.fillna(0)
# movie_ratings = movie_ratings.values

# best_score = None
# best_params = None

# for grid in list(ParameterGrid(param_grid)):
    
#     cv_score = []
#     print('Grid: ', grid)
#     for train_rating, valid_rating in test_valid_rating_split(movie_ratings):
#         mf = MF(**grid)
#         mf.fit(movie_ratings)
        
#         cv_score.append(mf.score(valid_rating))
#     cv_score = np.mean(cv_score)
    
#     if best_score is None or best_score > cv_score:
#         best_score = cv_score
#         best_params = grid
#         print('best score:', best_score)
#         print('best params:', best_params)

# print("GridSearch result:")
# print('best score:', best_score)
# print('best params:', best_params)

In [9]:
params = [
    {'max_iter': 100, 'n_factors': 10, 'reg':   1, 'solver': 'lbfgs'},
    {'max_iter': 200, 'n_factors': 10, 'reg':   1, 'solver': 'lbfgs'},
    {'max_iter': 100, 'n_factors': 20, 'reg':   1, 'solver': 'lbfgs'},
    {'max_iter': 200, 'n_factors': 20, 'reg':   1, 'solver': 'lbfgs'},
]

rating_columns = [col for col in train.columns if col.startswith('Rating')]
data = test
movie_ratings = data[rating_columns].fillna(0)
movie_ratings = movie_ratings.values

for param in params:
    
    mf = MF(**param)
    mf.fit(movie_ratings)
    
    estimate = mf.predict_full()
    
    estimate = pd.DataFrame(estimate, columns=rating_columns)
    estimate['ID'] = np.arange(1, len(test) + 1)
    estimate = estimate[['ID', 'Rating [Despicable Me]']]
    
    pred = estimate[['Rating [Despicable Me]']].values
    fn = 'nf_{}_reg_{}_solver_{}_iter_{}_test.csv'.format(
        param['n_factors'], param['reg'], param['solver'], param['max_iter'])
    estimate.to_csv(fn, index=False)
    
    estimate['Rating [Despicable Me]'] = np.ceil(pred)
    
    fn = 'nf_{}_reg_{}_solver_{}_iter_{}_test_ceil.csv'.format(
        param['n_factors'], param['reg'], param['solver'], param['max_iter'])
    estimate.to_csv(fn, index=False)
    
#     estimate['Rating [Despicable Me]'] = np.rint(pred)
#     fn = 'nf_{}_reg_{}_solver_{}_iter_{}_test_rint.csv'.format(
#          param['n_factors'], param['reg'], param['solver'], param['max_iter'])
    
    estimate.to_csv(fn, index=False)
    
    print('params:', param)
    print('score:', mf.score(movie_ratings))

params: {'max_iter': 100, 'n_factors': 10, 'reg': 1, 'solver': 'lbfgs'}
score: 1.0581264053391577
params: {'max_iter': 200, 'n_factors': 10, 'reg': 1, 'solver': 'lbfgs'}
score: 1.0595329538157798
params: {'max_iter': 100, 'n_factors': 20, 'reg': 1, 'solver': 'lbfgs'}
score: 1.0601753104769744
params: {'max_iter': 200, 'n_factors': 20, 'reg': 1, 'solver': 'lbfgs'}
score: 1.058647940492935


In [10]:
# params = [
# #     {'max_iter': 100, 'n_factors':  5, 'reg':   3, 'solver': 'lbfgs'},
# #     {'max_iter': 100, 'n_factors':  5, 'reg':   1, 'solver': 'lbfgs'},
# #    {'max_iter': 100, 'n_factors': 10, 'reg':   3, 'solver': 'lbfgs'},
#     {'max_iter': 100, 'n_factors': 10, 'reg':   1, 'solver': 'lbfgs'},
# #    {'max_iter': 100, 'n_factors': 20, 'reg':   3, 'solver': 'lbfgs'},
#     {'max_iter': 100, 'n_factors': 20, 'reg':   1, 'solver': 'lbfgs'},

# #     {'max_iter': 500, 'n_factors':  5, 'reg':   3, 'solver': 'bfgs'},
# #     {'max_iter': 500, 'n_factors':  5, 'reg':   1, 'solver': 'bfgs'},
# #     {'max_iter': 500, 'n_factors': 10, 'reg':   3, 'solver': 'bfgs'},
# #     {'max_iter': 500, 'n_factors': 10, 'reg':   1, 'solver': 'bfgs'},
# #     {'max_iter': 500, 'n_factors': 20, 'reg':   3, 'solver': 'bfgs'},
# #     {'max_iter': 500, 'n_factors': 20, 'reg':   1, 'solver': 'bfgs'},
# ]

# rating_columns = [col for col in train.columns if col.startswith('Rating')]
# data = pd.concat([train, test])
# movie_ratings = data[rating_columns].fillna(0)
# movie_ratings = movie_ratings.values

# for param in params:
    
#     mf = MF(**param)
#     mf.fit(movie_ratings)
    
#     estimate = mf.predict_full()
#     estimate = estimate[len(train):, :]
    
#     estimate = pd.DataFrame(estimate, columns=rating_columns)
#     estimate['ID'] = np.arange(1, len(test) + 1)
#     estimate = estimate[['ID', 'Rating [Despicable Me]']]
    
#     pred = estimate[['Rating [Despicable Me]']].values
#     fn = 'nf_{}_reg_{}_solver_{}_iter_{}_full.csv'.format(
#         param['n_factors'], param['reg'], param['solver'], param['max_iter'])
#     estimate.to_csv(fn, index=False)
    
#     estimate['Rating [Despicable Me]'] = np.ceil(pred)
    
#     fn = 'nf_{}_reg_{}_solver_{}_iter_{}_full_ceil.csv'.format(
#         param['n_factors'], param['reg'], param['solver'], param['max_iter'])
#     estimate.to_csv(fn, index=False)
    
# #     estimate['Rating [Despicable Me]'] = np.rint(pred)
# #     fn = 'nf_{}_reg_{}_solver_{}_iter_{}_test_rint.csv'.format(
# #          param['n_factors'], param['reg'], param['solver'], param['max_iter'])
    
#     estimate.to_csv(fn, index=False)
    
#     print('params:', param)
#     print('score:', mf.score(movie_ratings))